In [ ]:
import gradio as gr
from hybrid_search_unsplash import hybrid_search_main
# from hybrid_search__XMRec import hybrid_search_main

In [ ]:
with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            textbox = gr.Textbox(label="Text Query")
            btn = gr.Button("Search", scale=0)

        # number = gr.Number(label="Number of images to retrieve")
        
        with gr.Row():
            # Add input fields for vector and keyword search boost levels
            vector_boost = gr.Number(label="Vector Search Boost Level")
            bm25_boost = gr.Number(label="Keyword Search Boost Level")
        
        gallery = gr.Gallery(label="Similar images",
                             show_label=False,
                             elem_id="gallery",
                             columns=[6],
                             object_fit="cover",
                             height="1000px"
                             )

        
        # Include the new input fields in the button click function
        btn.click(fn=hybrid_search_main,
                  inputs=[textbox, vector_boost, bm25_boost],
                  outputs=gallery)

demo.launch()

In [ ]:
import pandas as pd

# Read the TSV files
photos_df = pd.read_csv('../../dataset/unsplash-research-dataset-lite-latest/photos.tsv000', sep='\t')
# Select specific columns
photos_df_test = photos_df[['photo_id','photo_url', 'photo_submitted_at', 'photo_description', 'photo_location_country', 'photo_location_city','ai_description' ]]


# Sort the dataframes by 'photo_id'
photos_df_test = photos_df_test.sort_values('photo_id')

# print(photos_df_test.head(10))


# Define the columns to merge
cols_to_merge = ['photo_submitted_at', 'photo_description', 'photo_location_country', 'photo_location_city', 'ai_description']

# Merge the columns
photos_df_test['meta_data'] = photos_df_test[cols_to_merge].apply(lambda x: ', '.join(x.dropna()), axis=1)

# Drop the original columns
photos_df_test = photos_df_test.drop(columns=cols_to_merge)

# Display the first 5 rows of the dataframe
print(photos_df_test.head(50))


import json

# Convert each row of the DataFrame to a dictionary and write to a JSON file
with open('meta_data.json', 'w', encoding='utf-8') as f:
    for _, row in photos_df_test.iterrows():
        json.dump(row.to_dict(), f, ensure_ascii=False)
        f.write('\n')


In [ ]:
import json
unsplash_data = "./meta_data.json"

def load_file(file_path):
    try:
        json_objects = []
        with open(file_path, "r") as json_file:
            for line in json_file:
                data = json.loads(line)
                json_objects.append(data)
        print("Done")
    finally:
        json_file.close()
    return json_objects

meta_data = load_file(unsplash_data)
meta_data = meta_data[2]["meta_data"]
meta_data